In [19]:
import os
import cv2
import random
from tqdm import tqdm
from shutil import copyfile
import shutil

In [3]:
%pwd

'/mnt/c/users/sv/source/Enhancement/Image-Enchancer/Experiments'

In [4]:
os.chdir("..")

In [5]:
dataset_path = './blur_dataset/'
blurred_path = './Dataset_blurred/'

In [6]:
# Ensure blurred dataset directory exists
os.makedirs(blurred_path, exist_ok=True)

In [7]:
# Function to apply random blur
def apply_random_blur(image):
    blur_type = random.choice(['average', 'gaussian', 'median'])
    if blur_type == 'average':
        return cv2.blur(image, (5, 5))
    elif blur_type == 'gaussian':
        return cv2.GaussianBlur(image, (5, 5), 0)
    elif blur_type == 'median':
        return cv2.medianBlur(image, 5)

In [8]:
# Get all image filenames
all_images = [f for f in os.listdir(dataset_path) if os.path.isfile(os.path.join(dataset_path, f))]
random.shuffle(all_images)

In [9]:
# Split images into blurred and non-blurred sets
blurred_images = all_images[:2500]
non_blurred_images = all_images[2500:]

In [10]:
# Apply blur to the selected images
for image_name in tqdm(blurred_images):
    img = cv2.imread(os.path.join(dataset_path, image_name))
    blurred_img = apply_random_blur(img)
    cv2.imwrite(os.path.join(blurred_path, image_name), blurred_img)

100%|██████████| 2500/2500 [05:52<00:00,  7.10it/s]


In [11]:
# Copy non-blurred images to the blurred dataset folder
for image_name in non_blurred_images:
    copyfile(os.path.join(dataset_path, image_name), os.path.join(blurred_path, image_name))

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_dir = './Dataset_blurred/train/'
val_dir = './Dataset_blurred/val/'

In [15]:
# Create directories if they don't exist
os.makedirs(train_dir + 'blurry/', exist_ok=True)
os.makedirs(train_dir + 'sharp/', exist_ok=True)
os.makedirs(val_dir + 'blurry/', exist_ok=True)
os.makedirs(val_dir + 'sharp/', exist_ok=True)

In [16]:
# Split into train and validation sets
blurred_images_train, blurred_images_val = train_test_split(blurred_images, test_size=0.2, random_state=42)
non_blurred_images_train, non_blurred_images_val = train_test_split(non_blurred_images, test_size=0.2, random_state=42)

In [17]:
# Move images to the respective directories
def move_images(image_list, destination_folder, label):
    for image_name in image_list:
        shutil.move(os.path.join(blurred_path, image_name), os.path.join(destination_folder, label, image_name))

In [20]:
move_images(blurred_images_train, train_dir, 'blurry')
move_images(non_blurred_images_train, train_dir, 'sharp')
move_images(blurred_images_val, val_dir, 'blurry')
move_images(non_blurred_images_val, val_dir, 'sharp')

In [28]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torch.nn.functional as F

In [22]:
# Data transforms
train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

In [23]:
# Datasets and dataloaders
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [24]:
# Simple CNN model (as before)
class BlurDetectionModel(nn.Module):
    def __init__(self):
        super(BlurDetectionModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [25]:
model = BlurDetectionModel()

In [26]:
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [29]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        labels = labels.float().unsqueeze(1)  # Convert labels to float for BCELoss
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
    
    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            labels = labels.float().unsqueeze(1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    print(f'Validation Loss: {val_loss/len(val_loader):.4f}')

Epoch [1/10], Loss: 0.6942


/home/vivek/.cache/pypoetry/virtualenvs/research-H8km-SSG-py3.10/lib/python3.10/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Validation Loss: 0.6928
Epoch [2/10], Loss: 0.6930
Validation Loss: 0.6931
Epoch [3/10], Loss: 0.6933
Validation Loss: 0.6932
Epoch [4/10], Loss: 0.6936
Validation Loss: 0.6921
Epoch [5/10], Loss: 0.6929
Validation Loss: 0.6919
Epoch [6/10], Loss: 0.6936
Validation Loss: 0.6930
Epoch [7/10], Loss: 0.6937
Validation Loss: 0.6925
Epoch [8/10], Loss: 0.6932
Validation Loss: 0.6932
Epoch [9/10], Loss: 0.6928
Validation Loss: 0.6920
Epoch [10/10], Loss: 0.6921
Validation Loss: 0.6896


In [30]:
%pwd

'/mnt/c/users/sv/source/Enhancement/Image-Enchancer'

In [34]:
from PIL import Image

# Define the image transform (same as used in training)
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

# Load and preprocess the image
image_path = './/Images//image_52.jpg'  # Replace with your image path
image = Image.open(image_path).convert('RGB')
image = transform(image)
image = image.unsqueeze(0)  # Add a batch dimension (batch size of 1)

# Set the model to evaluation mode
model.eval()

# Make a prediction 
with torch.no_grad():
    output = model(image)
    prediction = torch.round(output).item()  # 0 or 1 for blur detection

# Interpret the result
if prediction == 1:
    print("The image is blurry.")
else:
    print("The image is not blurry.")

The image is blurry.
